# Data Analysis of ports

## Summary statistics

In [27]:
folder_name = '90739-iuu-systems-project/'
import sys
sys.path.insert(1, sys.path[0].partition(folder_name)[0] + folder_name)
import pandas as pd
import difflib 

In [16]:
reefers = pd.read_csv('/Users/sebastiandodt/OneDrive/Uni/Carnegie Mellon University/Modules/2022 Fall/Systems Project/Coding/90739-iuu-systems-project/port_evaluation/data/proc/labels/classified_carriers_v1.csv')
path = '/Users/sebastiandodt/OneDrive/Uni/Carnegie Mellon University/Modules/2022 Fall/Systems Project/Coding/90739-iuu-systems-project/port_evaluation/data/xlsx/gfw_data-v2.xlsx'
port_visits = pd.read_excel(path, sheet_name='carrier_port_visit-v2')

In [18]:
path = '/Users/sebastiandodt/OneDrive/Uni/Carnegie Mellon University/Modules/2022 Fall/Systems Project/Coding/90739-iuu-systems-project/port_evaluation/data/input/World_Port_Index/WPI.csv'
wpi = pd.read_csv(path)

In [20]:
## pre-processing reefers

reefers['share_of_loitering'] = reefers['loitering']/(reefers['encounter']+reefers['loitering'])
reefers['more_than_100_encounters'] = reefers['loitering'] + reefers['encounter'] > 100

In [21]:
port_vessel_count = port_visits.groupby(['port.name', 'vessel.id'])['id'].count().reset_index()
port_vessel_with_label = port_vessel_count.merge(reefers, how='left', left_on='vessel.id', right_on='id', suffixes=('_port', '_carrier'))
share_of_loitering = port_vessel_with_label.groupby('port.name')['share_of_loitering'].mean().reset_index()
number_of_bad_visits = port_vessel_with_label.groupby('port.name')['label'].sum().reset_index()
share_of_bad_visits = port_vessel_with_label.groupby('port.name')['label'].mean().reset_index()
# ports_summary = port_visits_known_ships.merge(share_of_loitering, on='port.name', how='left')
ports_summary = share_of_loitering.merge(number_of_bad_visits, on='port.name', how='left')
ports_summary = ports_summary.merge(share_of_bad_visits, on='port.name', how='left', suffixes=('_absolute_count', '_avg'))

In [26]:
ports_summary.sort_values('label_absolute_count', ascending=False)

,port.name,share_of_loitering,label_absolute_count,label_avg
514,BUSAN,0.849673,692.0,0.837772
2127,SINGAPORE,0.915148,550.0,0.963222
1669,PANAMA CITY,0.962655,463.0,0.985106
606,COLON,0.964614,450.0,0.984683
1276,LAS PALMAS,0.968288,427.0,0.981609
...,...,...,...,...
1876,PUERTO QUEMCHI,0.004274,0.0,0.000000
1135,KAWAIHAE,NaN,0.0,NaN
488,BRINNON,0.500000,0.0,0.000000
490,BRIXHAM HARBOUR,NaN,0.0,NaN


In [24]:
wpi

,"INDEX_NO,N,16,6","REGION_NO,N,16,6","PORT_NAME,C,254","COUNTRY,C,254","LATITUDE,N,16,6","LONGITUDE,N,16,6","LAT_DEG,N,16,6","LAT_MIN,N,16,6","LAT_HEMI,C,254","LONG_DEG,N,16,6",...,"ELECREPAIR,C,254","PROVISIONS,C,254","WATER,C,254","FUEL_OIL,C,254","DIESEL,C,254","DECKSUPPLY,C,254","ENG_SUPPLY,C,254","REPAIRCODE,C,254","DRYDOCK,C,254","RAILWAY,C,254"
0,61090.0,61070.0,SHAKOTAN,RU,43.866667,146.833333,43.0,52.0,N,146.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,61110.0,61100.0,MOMBETSU KO,JP,44.350000,143.350000,44.0,21.0,N,143.0,...,NaN,NaN,Y,NaN,Y,NaN,NaN,C,NaN,NaN
2,61120.0,61100.0,ABASHIRI KO,JP,44.016667,144.283333,44.0,1.0,N,144.0,...,NaN,NaN,Y,NaN,Y,NaN,NaN,C,NaN,NaN
3,61130.0,61100.0,NEMURO KO,JP,43.333333,145.583333,43.0,20.0,N,145.0,...,NaN,Y,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN
4,61140.0,61100.0,HANASAKI KO,JP,43.283333,145.583333,43.0,17.0,N,145.0,...,NaN,NaN,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3664,60205.0,59850.0,SUIZHANG GANG,CN,40.066667,121.050000,40.0,4.0,N,121.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3665,44790.0,44360.0,BODRUM,TR,37.033333,27.433333,37.0,2.0,N,27.0,...,NaN,NaN,Y,Y,Y,NaN,NaN,C,NaN,NaN
3666,47110.0,47090.0,LAMU,KE,-2.266667,40.900000,2.0,16.0,S,40.0,...,NaN,NaN,Y,NaN,NaN,NaN,NaN,N,NaN,NaN
3667,60376.0,60315.0,GWANGYANG HANG,KR,34.850000,127.800000,34.0,51.0,N,127.0,...,NaN,Y,Y,Y,Y,NaN,NaN,B,M,NaN


In [57]:
## fuzzy matching
ports_summary['port_matched'] = ports_summary['port.name'].apply(lambda x: ''.join(difflib.get_close_matches(str(x), wpi['PORT_NAME,C,254'], n=1, cutoff=0.8)))

ports_matched = ports_summary.merge(wpi, how='left', left_on='port_matched', right_on='PORT_NAME,C,254')


In [59]:
print(ports_matched['PORT_NAME,C,254'].isna().sum())
ports_matched[['port.name','port_matched']].apply(lambda x: print(x))

1020
0         34103311
1         34103331
2         34122517
3         34122595
4         34123311
           ...    
2633        ZLARIN
2634      ZOLOTAYA
2635    ZONA COMUN
2636     ZONGULDAK
2637           ZUT
Name: port.name, Length: 2638, dtype: object
0                
1                
2                
3                
4                
          ...    
2633             
2634             
2635             
2636    ZONGULDAK
2637             
Name: port_matched, Length: 2638, dtype: object


port.name       None
port_matched    None
dtype: object

In [46]:
ports_summary['port.name'].apply(lambda x: difflib.get_close_matches(str(x), wpi['PORT_NAME,C,254']))

0       None
1       None
2       None
3       None
4       None
        ... 
2579    None
2580    None
2581    None
2582    None
2583    None
Name: port.name, Length: 2584, dtype: object

To-do:
- extract which loiterings happened before which ports visits